In [2]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [3]:
using Pkg

### Data

In [4]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [5]:
cell_demand = 0.001*164.98*(1.369*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

# D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [6]:
penalty = 10000 #ton/yr

10000

In [7]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [8]:
ipt = []
ipt = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount = ipt
# input_amount = round.(ipt, digits=2)

# MktV = price .* input_amount;  # ton

15-element Vector{Float64}:
  49670.291589732
  69512.470736544
  67696.846500888
  69382.78329114
 129687.44540400001
  72376.074329
   5239.896784
   1309.974196
  31111.887154999997
  60258.81301599999
 327493.549
   2258.5762
   1806.86096
 451715.24
 451715.24

----

In [70]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [71]:
#variables 
@variable(model, x[1:ncty, 1:nproc])            # x[i,k] production amount of product k at location i          
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0);   # y[i,j,k] ship product k from i to j 

In [62]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end


# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end


# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end


# LCA: As=f
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end


# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)


# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end

In [63]:
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 


############################
tranD = Vector{AffExpr}(undef, ncty)
tranS= Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end


###################################
Allo_soc = (proD+tranD) ./ emission_c .* Dsoc


#############
vec_S = proS + tranS
vec_D = proD + tranD
vec_Dsoc = Allo_soc;

In [64]:
UB = vec_S * 0.9
LB = vec_Dsoc * 1.1;

In [66]:
@objective(model, Max, sum(vec_D - LB));

In [68]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3166 rows, 14430 columns and 57630 nonzeros
Model fingerprint: 0x0c3942a5
Variable types: 13950 continuous, 480 integer (480 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [6e-02, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2491 rows and 12698 columns
Presolve time: 0.03s
Presolved: 675 rows, 1732 columns, 6156 nonzeros
Variable types: 1542 continuous, 190 integer (190 binary)
Found heuristic solution: objective 9746723.5568
Found heuristic solution: objective 1.059924e+07

Root relaxation: objective 1.236525e+07, 369 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth I

In [69]:
opt_x1 = JuMP.value.(x)
opt_y1 = JuMP.value.(y)
opt_delta1 = JuMP.value.(delta)
opt1 = JuMP.objective_value(model);

In [70]:
res_x = DataFrame(opt_x1, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries)

Row,country,Li,Co,Mn,Ni,cathode,graphite,PP,PE,Cu,Al,cell,PET,electronics,noncell,battery
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia,49670.3,7000.0,0.0,15382.8,0.0,0.0,5239.9,1309.97,31111.9,0.0,0.0,0.0,0.0,0.0,0.0
3,Brazil,0.0,0.0,0.0,0.0,0.0,52576.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Canada,0.0,3100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Chile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,China,0.0,2200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Colombia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Congo,0.0,36558.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# CSV.write("/Users/bourg/Desktop/eco_safe_1013_.csv", res_x) 

----

In [22]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [23]:
#variables 
@variable(model, x[1:ncty, 1:nproc])            # x[i,k] production amount of product k at location i          
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0);   # y[i,j,k] ship product k from i to j 

In [25]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end


# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end


# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end


# LCA: As=f
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end


# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)


# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end

In [26]:
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 


############################
tranD = Vector{AffExpr}(undef, ncty)
tranS= Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end


###################################
Allo_soc = (proD+tranD) ./ emission_c .* Dsoc


#############
vec_S = proS + tranS
vec_D = proD + tranD
vec_Dsoc = Allo_soc;

In [27]:
UB = vec_S * 0.9;

In [29]:
@objective(model, Min, sum(vec_D-UB));

In [30]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3166 rows, 14430 columns and 57630 nonzeros
Model fingerprint: 0x1aa858f2
Variable types: 13950 continuous, 480 integer (480 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [7e-03, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2491 rows and 12698 columns
Presolve time: 0.06s
Presolved: 675 rows, 1732 columns, 6156 nonzeros
Variable types: 1542 continuous, 190 integer (190 binary)
Found heuristic solution: objective 339709.17328
Found heuristic solution: objective -133110.9179

Root relaxation: objective -6.432280e+06, 109 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth 

In [31]:
opt_x2 = JuMP.value.(x)
opt_y2 = JuMP.value.(y)
opt_delta2 = JuMP.value.(delta)
opt2 = JuMP.objective_value(model);

In [32]:
res_x = DataFrame(opt_x2, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries)

Row,country,Li,Co,Mn,Ni,cathode,graphite,PP,PE,Cu,Al,cell,PET,electronics,noncell,battery
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,6200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia,1770.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brazil,2200.0,0.0,67696.8,67000.0,0.0,62376.1,5239.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Canada,500.0,3100.0,0.0,2382.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Chile,39000.0,0.0,0.0,0.0,0.0,0.0,0.0,1309.97,31111.9,0.0,0.0,2258.58,0.0,4.51715e5,0.0
6,China,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Colombia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Congo,0.0,66412.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Test

In [107]:
proD = (opt_x1 .* Matrix(regional_EF[:,2:end])) * ones(nproc,1);

In [108]:
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (opt_x1.*pro_sink)*ones(nproc,1);

In [109]:
tranD = zeros(ncty,1)
tranS = zeros(ncty,1)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(opt_y1[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end

In [110]:
Supply = proS + tranS
Demand = proD + tranD;

In [111]:
Dsoc_proc = (proD+tranD) ./ emission_c .* Dsoc;

In [112]:
UB = Supply * 0.9
LB = Dsoc_proc * 1.1;

In [113]:
opt2_sub = sum(Demand-UB)

5.697904982969201e6

In [114]:
opt2

-6.43227950298411e6

---

In [115]:
proD = (opt_x2 .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (opt_x2.*pro_sink)*ones(nproc,1)

tranD = zeros(ncty,1)
tranS = zeros(ncty,1)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(opt_y1[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end

In [116]:
Supply = proS + tranS
Demand = proD + tranD;

In [117]:
Dsoc_proc = (proD+tranD) ./ emission_c .* Dsoc;

In [118]:
UB = Supply * 0.9
LB = Dsoc_proc * 1.1;

In [119]:
opt1_sub = sum(Demand - Dsoc_proc)

9.70657480829237e6

In [120]:
opt1

1.23652462033327e7

---

In [130]:
opt2_sub, opt2

(5.697904982969201e6, -6.43227950298411e6)

In [129]:
eps = (opt2_sub - opt2) / 3

4.043394828651104e6

In [131]:
p1 = opt2 + eps

-2.388884674333006e6

In [132]:
p2 = opt2 + 2*eps

1.654510154318098e6

In [133]:
p3 = opt2 + 3*eps

5.697904982969202e6

---

In [148]:
model = Model(Gurobi.Optimizer)

#variables 
@variable(model, x[1:ncty, 1:nproc])            # x[i,k] production amount of product k at location i          
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0);   # y[i,j,k] ship product k from i to j 

Academic license - for non-commercial use only - expires 2023-11-27


In [149]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end


# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end


# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end


# LCA: As=f
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end


# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)


# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end

In [150]:
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 


############################
tranD = Vector{AffExpr}(undef, ncty)
tranS= Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
    tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
end


###################################
Allo_soc = (proD+tranD) ./ emission_c .* Dsoc


#############
vec_S = proS + tranS
vec_D = proD + tranD
vec_Dsoc = Allo_soc;

In [151]:
UB = vec_S * 0.9
LB = vec_Dsoc * 1.1;

In [152]:
@constraint(model, sum(vec_D-UB) == p3);

In [153]:
@objective(model, Max, sum(vec_D - LB));

In [154]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2207 rows, 13950 columns and 42270 nonzeros
Model fingerprint: 0x616edb05
Coefficient statistics:
  Matrix range     [4e-03, 3e+02]
  Objective range  [6e-02, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 2031 rows and 12498 columns
Presolve time: 0.02s
Presolved: 176 rows, 1452 columns, 4113 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.485e+03
 Factor NZ  : 3.072e+03 (roughly 1 MByte of memory)
 Factor Ops : 6.975e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Prim

In [155]:
res = []

Any[]

In [156]:
eps = (opt2_sub - opt2) / 5

2.4260368971906626e6

In [158]:
opt2 + 0*eps

-6.43227950298411e6

In [159]:
for i in 0:5
    
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc])                   
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0);   
    

    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
        end
    end

    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
        end
    end

    for i in 1:ncty
        for j in [e for e in 1:ncty if e != mkt_loc]
            @constraint(model, y[i,j,mkt_proc] == 0)
        end
    end

    for k in 1:nproc
        @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
    end

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)

    for k in cathode
        for j in 1:ncty
            @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
        end
    end

    for k in cell
        for j in 1:ncty
            @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
        end
    end

    for k in noncell
        for j in 1:ncty
            @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
        end
    end

    for k in battery
        for j in 1:ncty
            @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
        end
    end

    ######################
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 

    tranD = Vector{AffExpr}(undef, ncty)
    tranS= Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
        tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
    end

    Allo_soc = (proD+tranD) ./ emission_c .* Dsoc

    vec_S = proS + tranS
    vec_D = proD + tranD
    vec_Dsoc = Allo_soc
    
    UB = vec_S * 0.9
    LB = vec_Dsoc * 1.1
    
    @constraint(model, sum(vec_D-UB) == opt2 + i*eps)
    @objective(model, Max, sum(vec_D - LB))
    JuMP.optimize!(model)
    
    obj = JuMP.objective_value(model)
    push!(res, obj);
    
end

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2207 rows, 13950 columns and 42270 nonzeros
Model fingerprint: 0xf5c829b4
Coefficient statistics:
  Matrix range     [4e-03, 3e+02]
  Objective range  [6e-02, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 2031 rows and 12498 columns
Presolve time: 0.02s
Presolved: 176 rows, 1452 columns, 4113 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.485e+03
 Factor NZ  : 3.072e+03 (roughly 1 MByte of memory)
 Factor Ops : 6.975e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective 

In [160]:
res

6-element Vector{Any}:
 8.726499750204878e6
 1.0872084470252732e7
 1.14506631698731e7
 1.1938420006763976e7
 1.2256856913102599e7
 1.2365524077047165e7